In [13]:
from openai import OpenAI
import pandas as pd

with open('../api_key.txt', 'r') as file:
            key = file.read().strip()

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=key,
)

In [91]:
data = pd.read_csv("../data/1867_italy.csv").drop('Unnamed: 0', axis=1)

a_null_entry = data.iloc[314,:]
an_almost_correct_entry = data.iloc[639, :] # Milau
a_wrong_entry = data.iloc[60, :] # Should be: Lugo de Vicence, is: lamain
a_second_wrong_entry = data.iloc[1572, :]

entries = [a_null_entry, an_almost_correct_entry, a_wrong_entry, a_second_wrong_entry]

In [139]:
def query_chatgpt(query):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",)
    
    return(chat_completion.choices[0].message.content)

In [140]:
question = """
In the following string, which contains potential typos, I am looking for a city name.
I know the city is located in Italy so please mention an Italian city. 
Give me the English or Italian name of the city, no other output:"
"""

question_without_breaks = question.replace('\n', '')

In [144]:
responses = []

for i in data['text']:
    query = question_without_breaks + " " + i
    responses.append(query_chatgpt(query))


In [146]:
data['respones'] = responses

In [ ]:
question_italian = """
Nella seguente stringa, che contiene potenziali errori di battitura, sto cercando il nome di una città.
So che la città si trova in Italia, quindi si prega di indicare una città italiana. 
Forniscimi il nome italiano della città, nessun altro output:"
"""

In [148]:
data.to_csv('../data/1867_italy_chatgpt.csv', index=False)